In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install Augmentor
!pip install numpy 
!pip install Pillow 

In [2]:
import Augmentor
import os
import shutil
from PIL import Image
import numpy as np

Documentation on Augmentor can be found here: https://github.com/mdbloice/Augmentor

In [9]:
root = '/Users/MR ARCHITECT/MyFloorPlan/MyGenFloorPlan'

In [4]:
pathA = root + '/dataset/Original/A'
pathB = root + '/dataset/Original/B' 

In [5]:
p = Augmentor.Pipeline(pathA)
# Point to a directory containing ground truth data.
# Images with the same file names will be added as ground truth data
# and augmented in parallel to the original data.
p.ground_truth(pathB)

# Add operations to the pipeline:
p.rotate90(probability=0.25)
p.rotate270(probability=0.25)
p.flip_left_right(probability=0.25)
p.flip_top_bottom(probability=0.25)


Initialised with 104 image(s) found.
Output directory set to /Users/MR ARCHITECT/MyFloorPlan/MyGenFloorPlan//dataset/Original/A\output.

104 ground truth image(s) found.


In [6]:
IMAGE_COUNT = 300

p.sample(IMAGE_COUNT)

Processing <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=256x256 at 0x25F6F691550>: 100%|██████████| 300/300 [00:00<00:00, 661.47 Samples/s]


In [ ]:
# Wait a few minutes before continuing to make sure the output images get uploaded to Google Drive, since we'll be 
# copying and combining them in the next step

In [11]:
newPathA = root + '/dataset/Augmented/A/'
newPathB = root + '/dataset/Augmented/B/'
combinedPath = root + '/dataset/Augmented/Combined/'

In [ ]:
# !mkdir root + newPathA

In [13]:
# Iterate over the files generated by the Augmentor pipeline and copy them to a new directory
# The empty destination directory needs to exist before running this cell

# This is the default location that Augmentor will have saved the sampled images
os.chdir(pathA + '/output')


A_files = []
B_files = []

for filename in os.listdir('.'):
  if 'A_original' in filename:
      # guid will include the .png suffix
      guid = filename[-16:]
      A_files.append(guid)
      # shutil.copyfile(src, dst, *, follow_symlinks=True)
      dst = newPathA + guid
      shutil.copyfile(filename, dst)
  if '_groundtruth_' in filename:
      # guid will include the .png suffix
      guid = filename[-16:]
      B_files.append(guid)
      # shutil.copyfile(src, dst, *, follow_symlinks=True)
      dst = newPathB + guid
      shutil.copyfile(filename, dst)

print("A Filenames Found: ", A_files)
print("B Filenames Found: ", B_files)

A Filenames Found:  ['28baa078b5f6.png', 'a26fbdb59fd4.png', '513a77d31e42.png', '1924f67a75ea.png', '849604b45677.png', '3a6eef3e1d47.png', '34c0b1e63a86.png', 'afff91894ee7.png', '0afd7614dc34.png', '5c98b064bdda.png', '0706c911b03f.png', '09afd5954fef.png', '177083950b9a.png', '921075bfc8f9.png', '1ed930938c89.png', 'd1a8fbc80659.png', '3797f1d8965f.png', 'f05ba4f2066e.png', 'd83505f10d59.png', 'b67acdc0af46.png', 'e59e3d02d2bd.png', 'f19c469eb2e0.png', '223bb0ca3a3e.png', 'd91767b50313.png', '28df11ad180c.png', '1be9ea76e16b.png', 'c578017a2e2b.png', 'dcc39f5d1d0a.png', '64e13e043972.png', '5227a7590d88.png', '439f456e687b.png', '5471c8bb23a2.png', 'c6e480eea4c1.png', 'f83af31912e5.png', '07f7fd6e0c28.png', 'e69d731367ab.png', 'e0760f9f0a7e.png', '65ec5949b296.png', 'bdf2e1bf415e.png', '5215b84a4f80.png', '55b32664d220.png', '00310e95d619.png', '465c536d1b14.png', '8786b29baa09.png', 'd70fa2af0f67.png', '89247caa9a67.png', '18fd149b0c40.png', 'de9b21bdd55b.png', 'f30df568b750.png',

In [14]:
# Combine Augmented Images into the pair format that Pix2Pix expects
os.chdir(newPathA)
for filename in os.listdir('.'):
  im1 = Image.open(filename)
  im2 = Image.open(newPathB + filename)

  # Combine images horizontally using PIL
  dst = Image.new('RGB', (512, 256))
  dst.paste(im1, (0, 0))
  dst.paste(im2, (im1.width, 0))
  dst.save(combinedPath + filename)

In [16]:
# Train/Test Split
# Takes the files in '../Augmented/Combined/' and randomly splits them into './Combined/Train' and './Combined/Test' directories
os.chdir(combinedPath)
test_dir = combinedPath + 'Test/'
train_dir = combinedPath + 'Train/'

files = os.listdir(combinedPath)
ratio = 0.15

for f in files:
    if '.png' in f:
      if np.random.rand(1) < ratio:
          shutil.copyfile(combinedPath + f, test_dir + f)
      else:
          shutil.copyfile(combinedPath + f, train_dir + f)

In [17]:
# Check that files exist in new location:

# os.chdir(pathA+ '/output')
os.chdir(train_dir)
print(len([name for name in os.listdir('.') if os.path.isfile(name)]))

261
